<a href="https://colab.research.google.com/github/Ngum12/study-fine-tunnning_projects/blob/main/Copy_of_Untitled9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU unsloth transformers datasets accelerate huggingface_hub wandb
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.1/464.1 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Step 2: Authentication
from huggingface_hub import notebook_login
notebook_login()

In [3]:
# Step 3: Initialize Model
from unsloth import FastLanguageModel

max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.8: Fast Llama patching. Transformers: 4.48.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.9k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [4]:
# Step 4: Prepare LoRA Configuration
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=3407,
)

Unsloth 2025.1.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
# Step 5: Load and Format Dataset
from datasets import load_dataset

dataset = load_dataset(
    "FreedomIntelligence/medical-o1-reasoning-SFT",
    "en",
    split="train[:500]",
    trust_remote_code=True
)

EOS_TOKEN = tokenizer.eos_token

def format_instruction(sample):
    return {
        "text": f"""Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{sample['Question']}

### Response:
<think>
{sample['Complex_CoT']}
</think>
{sample['Response']}{EOS_TOKEN}"""
    }

dataset = dataset.map(format_instruction)

README.md:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

medical_o1_sft.json:   0%|          | 0.00/74.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25371 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [6]:
# Step 6: Configure Training
from transformers import TrainingArguments
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="wandb",
    ),
)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [7]:
# Step 7: Start Training
import wandb
wandb.init(project="Medical-CoTAI-Finetuning")

trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dieudonnen450 (dieudonnen450-african-leadership-academy) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 500 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
10,1.913800
20,1.468400
30,1.408700
40,1.315300
50,1.351000
60,1.321200


TrainOutput(global_step=60, training_loss=1.463069995244344, metrics={'train_runtime': 1174.3042, 'train_samples_per_second': 0.409, 'train_steps_per_second': 0.051, 'total_flos': 1.792773518721024e+16, 'train_loss': 1.463069995244344, 'epoch': 0.96})

In [8]:
# Step 8: Save Model
model.save_pretrained("fine_tuned_medical_coTAI")
#model.push_to_hub("your-username/fine_tuned_medical_coTAI")


In [9]:
# Add this at the very top of your notebook
import torch

# Then add this to your existing code
def generate_response(question):
    FastLanguageModel.for_inference(model)

    prompt = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
You are a medical expert. Answer the following medical question.

### Question:
{question}

### Response:
<think>"""

    inputs = tokenizer(prompt, return_tensors="pt", padding=True).to("cuda")

    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=1200,
        pad_token_id=tokenizer.eos_token_id,
        temperature=0.7,
        do_sample=True,
        use_cache=True,
    )

    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return full_response.split("### Response:")[-1].strip()

# Test with sample question
sample_question = "give python code for a medical website?"

# Set model to evaluation mode
model.eval()
with torch.inference_mode():  # Now this will work
    print(generate_response(sample_question))

<think>
Alright, let's see what we're dealing with here. I need to create a medical website using Python, and it should be accessible to both doctors and patients. First, I need to figure out what the website should include. Doctors would probably want a login page where they can access patient records and other medical tools. Patients might want a way to view their test results, schedule appointments, or request medical advice.

Hmm, I need a user-friendly interface. Maybe I can use a Flask framework because it's pretty simple and commonly used in the medical field. It's also good for building web apps quickly. Alright, Flask it is.

Now, let's think about the structure of the website. For doctors, a login screen seems essential. I should set up a login page that verifies their credentials using a secure method. Oh, I could use OAuth for this, but maybe I'll just stick with a basic username and password check for simplicity.

Next, the home page for doctors should have a dashboard. It

In [ ]:
model.save_pretrained("DeepSeek-R1-Medical-COT")
tokenizer.save_pretrained("DeepSeek-R1-Medical-COT")

model.save_pretrained_merged("DeepSeek-R1-Medical-COT", tokenizer, save_method="merged_16bit")


In [ ]:
!zip -r DeepSeek-R1-Medical-COT.zip DeepSeek-R1-Medical-COT/


In [ ]:
!git config --global user.email "d.ngum@alustudent.com"
!git config --global user.name "Ngum12"

!git clone https://github.com/Ngum12/Fine-Tune_Data-with-DeepSeek.git


In [ ]:
!mv DeepSeek-R1-Medical-COT.zip Fine-Tune_Data-with-DeepSeek/

In [ ]:
%cd Fine-Tune_Data-with-DeepSeek

In [ ]:
!git add DeepSeek-R1-Medical-COT.zip
!git commit -m "Added fine-tuned DeepSeek-R1-Medical-COT model"
!git push origin main
